In [85]:
from importlib import reload
from load_cluster_data import load_cluster_data
from pca_kmeans_init import pca_kmeans_init
from betabinomo_mix_singlecells import *
import betabinomo_mix_singlecells
reload(betabinomo_mix_singlecells)
import torch
import sklearn.manifold 
import plotnine as p9
import time
# indicate plot should be small 4 by 4
import plotnine as p9
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap, geom_violin, theme, element_blank, geom_text
import plotnine
from tqdm import tqdm
plotnine.options.figure_size = (4, 4)

### Settings and Load data

In [2]:
input_file = '/gpfs/commons/scratch/kisaev/ss_tabulamuris_test/Leaflet/clustered_junctions_noanno.txt_anno_free_50_500000_10_5_0.1_single_cell.h5'

torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

MAKE_PCA_TSNE = True

float_type = { 
    "device" : device, 
    "dtype" : torch.float, # save memory
}

hypers = {
    "eta" : 1., 
    "alpha_prior" : 1., # karin had 0.65 
    "pi_prior" : 1.
}

cpu


In [3]:
final_data, coo_counts_sparse, coo_cluster_sparse, cell_ids_conversion, junction_ids_conversion = load_cluster_data(
    input_file) 
N = coo_cluster_sparse.shape[0]
J = coo_cluster_sparse.shape[1]

cell_index_tensor, junc_index_tensor, my_data = betabinomo_mix_singlecells.make_torch_data(final_data, **float_type)

['Brain_Myeloid_macrophage_pseudobulk'
 'Brain_Non-Myeloid_Bergmann_glial_cell_pseudobulk'
 'Brain_Non-Myeloid_astrocyte_pseudobulk'
 'Brain_Non-Myeloid_brain_pericyte_pseudobulk'
 'Brain_Non-Myeloid_endothelial_cell_pseudobulk'
 'Brain_Non-Myeloid_neuron_pseudobulk'
 'Brain_Non-Myeloid_oligodendrocyte_precursor_cell_pseudobulk'
 'Brain_Non-Myeloid_oligodendrocyte_pseudobulk']
3462
3462
63257
63257
The number of junctions in the data is:  63257
The number of cells in the data is:  3462
The number of cell types in the data is:  8


/gpfs/commons/home/kisaev/leafcutter-sc/src/beta-binomial-lda/betabinomo_mix_singlecells.py:299: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)


In [4]:
num_trials = 2 # should also be an argument that gets fed in
num_iters = 30 # should also be an argument that gets fed in
K = 5

# loop over the number of trials (for now just testing using one trial but in general need to evaluate how performance is affected by number of trials)
reload(betabinomo_mix_singlecells)

start_time = time.time()
results = [ betabinomo_mix_singlecells.calculate_CAVI(K, my_data, float_type, hypers, init_labels = None, num_iterations = num_iters) 
           for t in range(num_trials) ]

# write the above line use fstring
print(f"This took {time.time() - start_time} seconds")

Initialize VI params
Got the initial ELBO ^
ELBO converged @ -1678499072.0  CAVI iteration #  18  complete
Initialize VI params
Got the initial ELBO ^
ELBO converged @ -1678051968.0  CAVI iteration #  30  complete
This took 15.786423921585083 seconds


In [5]:
# extract PHI_f from every trial in num_trials
all_iters_PHI_f = [ result[3] for result in results ]
i = 0

# Create an empty list to store DataFrames from each iteration
dfs_list = []

for PHI_var in all_iters_PHI_f:

    probability_tensor = PHI_var

    # Create an array with cell IDs (e.g., cell_0, cell_1, ..., cell_(N-1))
    cell_ids = np.arange(probability_tensor.shape[0])
    cell_ids = [cell_id for cell_id in cell_ids]

    # Get the cluster IDs for each cell based on the maximum probability
    cluster_ids = np.argmax(probability_tensor, axis=1)

    # Create a DataFrame with the cell_id, cluster_id, and probability columns
    df = pd.DataFrame({"cell_id": cell_ids, "cluster_id": cluster_ids})

    # Add column with iteration number
    df["iteration"] = i
    i += 1
    # Append the DataFrame to the list
    dfs_list.append(df)

# Concatenate all the DataFrames into a single DataFrame
concatenated_df = pd.concat(dfs_list, ignore_index=True)

# Display the concatenated DataFrame
print(concatenated_df.head())

   cell_id  cluster_id  iteration
0        0           3          0
1        1           3          0
2        2           3          0
3        3           3          0
4        4           3          0


In [14]:
from itertools import combinations
all_cell_pairs = list(combinations(concatenated_df.cell_id.unique(), 2))
grouped_data = concatenated_df.groupby(['cell_id', 'iteration'])['cluster_id'].unique()

In [116]:
import pandas as pd
import numpy as np
from itertools import groupby

# Assuming you have a DataFrame called 'concatenated_df' with 'cell_id', 'iteration', and 'cluster_id' columns

# Preprocess the DataFrame to get the unique cluster IDs for each 'cell_id' and 'iteration'
preprocessed_data = concatenated_df.groupby(['cell_id', 'iteration'])['cluster_id'].unique()

# Create a dictionary to store the unique cluster IDs for each cell
unique_clusters_by_cell = {cell: preprocessed_data[cell] for cell in preprocessed_data.index.get_level_values('cell_id').unique()}

# Initialize an empty list to store the results
results = []

# Loop through all the cell pairs in 'all_cell_pairs'
for cell_1, cell_2 in tqdm(all_cell_pairs, desc="Processing pairs"):
    # Check if both cells have iterations
    if cell_1 in unique_clusters_by_cell and cell_2 in unique_clusters_by_cell:
        # Get the unique cluster IDs for both cells
        cluster_ids_cell_1 = unique_clusters_by_cell[cell_1]
        cluster_ids_cell_2 = unique_clusters_by_cell[cell_2]

        # Check if the cluster IDs are the same in both cells' iterations
        is_same_cluster = (cluster_ids_cell_1 == cluster_ids_cell_2).all()

        # Append the result to the list
        results.append(((cell_1, cell_2), is_same_cluster))

# Display the results
for cell_pair, is_same_cluster in results:
    print(f"Cell pair: {cell_pair}, Same cluster assignments: {is_same_cluster}")

Processing pairs:   7%|▋         | 402402/5990991 [01:36<22:15, 4183.31it/s]


KeyboardInterrupt: 

In [87]:
# 3462 unique cells in the concatenated_df
results = list()

for cell in tqdm(concatenated_df.cell_id.unique()):
    # get all pairs from all_cell_pairs that begin with cell
    cell_pairs = [pair for pair in all_cell_pairs if pair[0] == cell]
    # for each subpair in cell_pairs, check if they get assigned together across iterations in concatenated_df
    for subpair in cell_pairs:
        # get data 
        test=concatenated_df[concatenated_df["cell_id"].isin([subpair[0], subpair[1]])].groupby(["iteration"]).cluster_id.unique()
        # check if in both iterations, the length is 1 (that means they are in the same cluster)
        res = test.apply(lambda x: len(x) == 1).unique()
        # append subpair and res to a list
        results.append((subpair, res))

  0%|          | 0/3462 [00:00<?, ?it/s]

  0%|          | 9/3462 [01:01<6:30:24,  6.78s/it]


KeyboardInterrupt: 

In [ ]:
best = np.argmax([ g[-1][-1] for g in results ]) # final ELBO
ALPHA_f, PI_f, GAMMA_f, PHI_f, elbos_all = results[best]
elbos_all = np.array(elbos_all)
plt.plot(elbos_all[1:]); plt.show()

In [ ]:
juncs_probs = ALPHA_f / (ALPHA_f+PI_f)    
plt.hist(juncs_probs.cpu().numpy().flatten(), 20); plt.show()

In [ ]:
PHI_f.shape

In [ ]:
theta_f_plot = pd.DataFrame(PHI_f.cpu().numpy())
theta_f_plot['cell_id'] = cell_ids_conversion["cell_type"].to_numpy()
theta_f_plot_summ = theta_f_plot.groupby('cell_id').mean()
print(theta_f_plot_summ)

In [ ]:
theta = GAMMA_f / GAMMA_f.sum()
theta

In [ ]:
# How much each cluster is used in each "cell state"
#latent proportions describe the general prevalence of each cluster in the datasetN

theta = GAMMA_f / GAMMA_f.sum()
theta = theta.cpu().numpy()
theta_sorted = np.sort(theta)
plt.bar(np.arange(K)+1,theta_sorted[::-1]); plt.show()

In [ ]:
to_keep = theta > 0.01

x = PHI_f.cpu().numpy()
x = x[:,to_keep]
#x -= x.mean(1,keepdims=True)
#x /= x.std(1,keepdims=True)
_ = plt.hist(x.flatten(),100)

In [ ]:
PHI_f

In [ ]:
x.shape

In [ ]:
ct = pd.crosstab( cell_ids_conversion["cell_type"], x.argmax(axis=1) )
ct

In [ ]:
ct = pd.crosstab( cell_ids_conversion["cell_type"], x.argmax(axis=1) )
ct_np = ct.to_numpy()
ct_np

In [ ]:
ct_np = ct_np / ct_np.sum(1, keepdims=True) # normalize cell-type counts
ct_np = ct_np / ct_np.sum(0, keepdims=True)

ct.iloc[:,:] = ct_np

ax = plt.figure(figsize=[10,8])
sns.clustermap(ct, dendrogram_ratio=0.15, vmin = None, figsize=(12,6), annot = False)

In [ ]:
juncs_probs

In [ ]:
juncs_probs_df = pd.DataFrame(juncs_probs, columns = range(K))
# add "cell_state" to each column name 
juncs_probs_df.columns = ["cell_state_" + str(col) for col in juncs_probs_df.columns]
juncs_probs_df["junction_id_index"] = junction_ids_conversion.junction_id_index.values
# convert to juncs_probs to pandas dataframe and calculate mean and std across cell states/topics
juncs_probs_df["junction_id"] = junction_ids_conversion.junction_id.values

def plot_juncObsUsage(junc_index):

    # print junction ID using junction_ids_conversion
    print(junction_ids_conversion[junction_ids_conversion["junction_id_index"] == junc_index])
    junc_id = junction_ids_conversion[junction_ids_conversion["junction_id_index"] == junc_index].junction_id.values[0]

    # get data for just junc_index 
    junc_dat=final_data[final_data.junction_id_index==junc_index]
    print(junc_dat.cell_type.value_counts())

    # make violin plot for junc_dat junction usage ratio coloured by cell_type and rotate plot 90 degrees
    plot = ggplot(junc_dat, aes(x='cell_type', y='juncratio', fill="cell_type")) + geom_violin() + geom_point() + plotnine.labels.ggtitle(junc_id) + plotnine.coords.coord_flip() 

    # add number of cells in each cell_type to plot 
    print(plot)

def plot_juncProbs(junc_index):
    
    # print junction ID using junction_ids_conversion
    print(junction_ids_conversion[junction_ids_conversion["junction_id_index"] == junc_index])
    junc_id = junction_ids_conversion[junction_ids_conversion["junction_id_index"] == junc_index].junction_id.values[0]
    
    # get data for just junc_index 
    junc_dat=juncs_probs_df[juncs_probs_df.junction_id_index==junc_index]
    junc_dat = junc_dat.melt().iloc[0:K]
    junc_dat.value = junc_dat.value.astype(float)
    # make violin plot for junc_dat junction usage ratio coloured by cell_type
    # don't print x-axis tick labels 
    plot = ggplot(junc_dat, aes(x='variable', y='value')) + geom_point() + theme(axis_text_x=element_blank())
    print(plot)

In [ ]:
# calculate sd deviation for each junction for cell states 0 to 19 
juncs_probs_df["sd"] = juncs_probs_df.iloc[:,0:K].std(axis=1)


In [ ]:
# write function that takes in cell state and returns top 10 junctions with the highest difference with all other K-1 cell states
def top10_juncs(cell_state):
    # for each junction get the difference between cell_state and all other cell states not including cell_state
    # return top 10 junctions with highest difference
    no_ref=juncs_probs_df[juncs_probs_df.columns[~juncs_probs_df.columns.isin([cell_state, "junction_id_index", "junction_id", "sd"])]]
    juncs_probs_df["diff"] = juncs_probs_df[cell_state] - no_ref.mean(axis=1)
    top10 = juncs_probs_df.sort_values(by="diff", ascending=False).head(10)
    return(top10.junction_id_index.values)

    # think of actually using the distributions... use the full beta distribution via KL divergence... (pairwise)
    # are the distributions across cell states for junctions more different than if they were coming from the same cell state

In [ ]:
def log_beta(a, b):
    return torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a + b)

def score(a, b):
    return log_beta(a,b).sum() - log_beta(a.sum(), b.sum())

In [ ]:
juncs_probs[0, [0,9]]

In [ ]:
# get likelihood ratio/bayes factor score for ALL junctions 
# let's compare just state 1 and 2 

scores_all_juncs = []
for junc_index in range(juncs_probs.shape[0]):
    a = ALPHA_f[junc_index, [0,9]]
    b = PI_f[junc_index, [0,9]]
    scores_all_juncs.append(score(a, b).item())

In [ ]:
# turn scores_all_juncs into dataframe and add junction_id_index as a column
scores_all_juncs_df = pd.DataFrame(scores_all_juncs, columns = ["score"])
scores_all_juncs_df["junction_id_index"] = junction_ids_conversion.junction_id_index.values

In [ ]:
scores_all_juncs_df.head()

In [ ]:
scores_all_juncs_df.sort_values(by="score", ascending=False).head(10)

In [ ]:
juncs_test=scores_all_juncs_df.sort_values(by="score", ascending=False).head(10).junction_id_index.values
juncs_test

In [ ]:
# make histogram of scores
scores_all_juncs_df.hist(column="score", bins=30)

In [ ]:
#top10juncs_state1 = top10_juncs("cell_state_9")
#top10juncs_state1

In [ ]:
# for each junction in top10juncs_state1, run plot_juncObsUsage and plot_juncProbs
for junc in juncs_test:
    plot_juncObsUsage(junc)
    plot_juncProbs(junc)